In [1]:
import json
import pandas as pd
from collections import defaultdict
import os

In [2]:
def ava_json(failinimi):
    with open(failinimi, "r", encoding = "UTF-8") as f:
        return json.load(f)

In [3]:
df = pd.read_csv ("../minikorpus/skoorid.csv")

In [4]:
df_drop = df.drop(["SKOORID", "VASTAJAID", "STD"], axis = 1)

In [5]:
df_form = df_drop[df["DIMENSIOON"] == "form"].drop("DIMENSIOON", axis = 1).fillna(0)
df_spont = df_drop[df["DIMENSIOON"] == "spont"].drop("DIMENSIOON", axis = 1).fillna(0)

In [7]:
sonastik = defaultdict(lambda: dict())

for file in df_form["FAIL"]:
    sonastik[file] = ava_json(os.path.join("../minikorpus/meta/", file))
    sonastik[file] = {**sonastik[file], **ava_json(os.path.join("../minikorpus/sõnad/", file))}
    sonastik[file]["formaalsus"] = list(df_form[df_form["FAIL"] == file]["KESKMINE SKOOR"].items())[0][1]/3
    sonastik[file]["spontaansus"] = list(df_spont[df_spont["FAIL"] == file]["KESKMINE SKOOR"].items())[0][1]/3

In [8]:
df_uus = pd.DataFrame.from_dict(sonastik, orient="index")

In [9]:
df_uus = df_uus.replace(-1, 0).drop(["tajuverbide_osaarv", "korduvate_juppide_arv"], axis = 1)

In [10]:
df_uus.to_csv("../minikorpus/Randmestik.csv")